In [ ]:
import cv2
import os
import glob
import numpy as np
import torch
import torch.nn as nn
device = torch.device('cpu')
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset, DataLoader
import math

In [ ]:
# A function to load datasets and define labels, image size, image name, and classes 

def load_train(train_path, image_size, classes):
    images = []
    labels = []
    img_names = []
    cls = []
    
    for fields in classes:   
        index = classes.index(fields)
        path = os.path.join(train_path, fields, '*g')
        files = glob.glob(path)
        for fl in files:
            image = cv2.imread(fl)
            image = cv2.resize(image, (image_size, image_size),0,0, cv2.INTER_LINEAR)
            image = image.astype(np.float64)
            image = np.multiply(image, 1.0 / 255.0)
            image = np.asarray(image).transpose(-1, 0, 1) 
            images.append(image)
            label = np.zeros(len(classes))
            label[index] = 1.0
            labels.append(label)
            flbase = os.path.basename(fl)
            img_names.append(flbase)
            cls.append(fields)
    images = np.array(images)
    labels = np.array(labels)
    img_names = np.array(img_names)
    cls = np.array(cls)

    return images, labels, img_names, cls

# To load training data

images, labels, img_names, cls = load_train('C:\\Users\\nekra\\OneDrive\\Desktop\\Courses\\ML\\data\\training_data', 332, ['pembroke', 'cardigan'])

# Building a class for creating dataset

class DataSet(Dataset):

    def __init__(self):

        
        self.n_samples = labels[0].shape
        
        self.x_data = torch.from_numpy(images).float() 

        y_data = torch.from_numpy(labels).float() 

        self.y_data = torch.max(y_data, 1)[1]

    def __getitem__(self, index):

        return self.x_data[index], self.y_data[index]

    def __len__(self):

        return self.n_samples
    
# Create dataset as an instance of DataSet class 

dataset = DataSet()
images = dataset[0:][0]
labels = dataset[0:][1]
images = images.to(device)
labels = labels.to(device)
labels

In [4]:
# Building the CNN model as a class

class ConvNet(nn.Module):

    def __init__(self):

        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3,stride=1, padding=1)
        
        self.batchnorm1 = nn.BatchNorm2d(32)

        self.pool = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=0)
        
        self.batchnorm2 = nn.BatchNorm2d(32)        
       
        
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3,stride=1, padding=0)
        
        self.batchnorm3 = nn.BatchNorm2d(64)

        self.fc1 = nn.Linear(64 * 40 * 40, 128)

        self.fc2 = nn.Linear(128, 2)        



    def forward(self, x):

        x = self.conv1(x)
        
        x = self.batchnorm1(x)      
        
        x = self.pool(F.relu(x))  
        
        x = self.conv2(x)
        
        x = self.batchnorm2(x)
        x = self.pool(F.relu(x))  
        
        x = self.conv3(x)
        
        x = self.batchnorm3(x)           
        
        x = self.pool(F.relu(x))  

        x = x.view(-1, 64 * 40 * 40)            

        x = F.relu(self.fc1(x))          
        
 
        x = self.fc2(x)                 
       
        return x
    
# 

model = ConvNet().to(device)

model = model.float()

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.0000001)

outputs = model(images)
loss = criterion(outputs, labels)
print(loss)
print(outputs)


tensor(0.7055, grad_fn=<NllLossBackward>)
tensor([[ 5.8690e-02,  2.1986e-02],
        [ 3.4820e-01, -3.7762e-01],
        [-1.6224e-01, -3.7263e-01],
        [ 1.7714e-01, -2.2895e-01],
        [ 1.0501e-01, -1.9188e-01],
        [ 5.8632e-02, -1.2829e-01],
        [ 1.3845e-01, -1.7502e-01],
        [ 1.0240e-01, -2.0679e-01],
        [-2.6728e-02,  1.2560e-02],
        [ 2.4330e-01, -3.8231e-02],
        [ 8.2974e-02, -9.6970e-02],
        [ 2.6276e-01, -2.2196e-01],
        [ 7.7621e-02, -2.8238e-01],
        [-7.9395e-02, -2.2382e-01],
        [ 6.1477e-02, -2.8876e-01],
        [-1.6678e-02,  4.7739e-03],
        [ 8.5018e-02, -2.2754e-01],
        [-5.2141e-02, -1.7303e-01],
        [ 2.5287e-01, -4.1793e-01],
        [-8.5493e-02, -4.4496e-02],
        [-3.2562e-02, -9.1179e-02],
        [ 2.9755e-01, -3.7160e-01],
        [ 2.6199e-01, -2.2740e-01],
        [ 5.7689e-02, -1.5750e-01],
        [ 1.5841e-01, -1.7958e-01],
        [ 3.2516e-01,  3.3876e-02],
        [ 3.1033e-01, 

In [ ]:
# To Iterat in the CNN model 

for epoch in range(1000):
 for i in range(73):
# Forward pass
  optimizer.zero_grad()
  output , classes = dataset[i*4:4*i+4]

  output = model(output)

  loss = criterion(output, classes)


 # Backward and optimize
  
  

  loss.backward()

  optimizer.step()
        
print(loss)

PATH = './cifar_net.pth'
torch.save(model.state_dict(), PATH)


In [ ]:
# Building the model on the images
model = ConvNet()
model.load_state_dict(torch.load(PATH))
output = model(images)
output

In [ ]:
# Getting the output of the model on train dataset

output = torch.softmax(output, dim = 0)

output  

with torch.no_grad():
    
 _, predicted = torch.max(output, 1)
 
predicted  


In [ ]:
# Calculating the accuracy of the model 

def accuracy(labels,predicted):
 count = 0
 for i in range(len(labels)):
  if labels[i]==predicted[i]:
   count = count + 1
  acc = (count/len(labels))*100
 return acc 

print(accuracy(labels,predicted))
     

  

In [6]:
# load and create test dataset

images, labels, img_names, cls = load_train('C:\\Users\\nekra\\OneDrive\\Desktop\\Courses\\ML\\data\\testing_data', 332, ['pembroke', 'cardigan'])


dataset = DataSet()


test_images = dataset[0:][0]
test_labels = dataset[0:][1]


print(test_images)
test_labels



tensor([[[[0.3255, 0.2902, 0.2667,  ..., 0.2235, 0.0745, 0.0353],
          [0.2627, 0.2275, 0.2078,  ..., 0.2549, 0.0353, 0.1255],
          [0.2588, 0.2196, 0.2000,  ..., 0.3373, 0.0118, 0.0667],
          ...,
          [0.5333, 0.1333, 0.2863,  ..., 0.2902, 0.3216, 0.3765],
          [0.3529, 0.1725, 0.2118,  ..., 0.3373, 0.3804, 0.4510],
          [0.4314, 0.5843, 0.4784,  ..., 0.5059, 0.5569, 0.6392]],

         [[0.4118, 0.3765, 0.3529,  ..., 0.2980, 0.1490, 0.1098],
          [0.3490, 0.3137, 0.2941,  ..., 0.3294, 0.1098, 0.2078],
          [0.3451, 0.3059, 0.2863,  ..., 0.4000, 0.0353, 0.1569],
          ...,
          [0.6157, 0.2118, 0.3569,  ..., 0.3255, 0.3569, 0.4118],
          [0.4353, 0.2510, 0.2784,  ..., 0.3725, 0.4157, 0.4863],
          [0.5137, 0.6627, 0.5451,  ..., 0.5412, 0.5922, 0.6745]],

         [[0.5216, 0.4824, 0.4627,  ..., 0.4431, 0.2941, 0.2549],
          [0.4588, 0.4235, 0.4039,  ..., 0.4745, 0.2549, 0.3490],
          [0.4549, 0.4157, 0.3961,  ..., 0

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
test_output = torch.softmax(test_output, dim = 0)

test_output  

with torch.no_grad():
    
 _, test_predicted = torch.max(test_output, 1)
 
test_predicted  



In [ ]:
# Getting the output of the model on test dataset
def accuracy(labels,predicted):
 count = 0
 for i in range(len(labels)):
  if labels[i]==predicted[i]:
   count = count + 1
  acc = (count/len(labels))*100
 return acc 

print(accuracy(test_labels,test_predicted))
     

  
        
